In [2]:
# Test Run

sample = 1

for image_test_saving in range(1):
    # sample = image_test_saving  # TEST, DELETE
    # Get Image
    image = 'images/sample' + str(sample) + '.tif'  # For OPERATION
    # image = 'output.png'  # For TESTING
    print(image)

    # Libraries
    import cv2
    import numpy as np
    from PIL import Image, ImageFilter

    # Contrast Increase
    img = cv2.imread(image, 1)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12,12)) # Change numbers as necessary
    cl = clahe.apply(l_channel)
    limg = cv2.merge((cl,a,b))
    enhanced_image = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    result = np.hstack((img, enhanced_image))
    cv2.imwrite('contrast.png', result)
    cv2.imwrite('s0.png', enhanced_image)

    # Initial Noise Reduction
    image = cv2.imread("s0.png")
    image = cv2.GaussianBlur(image,(5,5),0)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    se=cv2.getStructuringElement(cv2.MORPH_RECT , (8,8))
    bg=cv2.morphologyEx(image, cv2.MORPH_DILATE, se)
    out_gray=cv2.divide(image, bg, scale=255)
    cv2.imwrite('s1.png',out_gray)

    # Script Noise Reduction
    image = cv2.imread('s1.png')
    image = cv2.GaussianBlur(image,(9,9),0)
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.equalizeHist(image)
    img_blur = cv2.GaussianBlur(image, (5, 5), 0)
    edges = cv2.Canny(img_blur, 0, 20)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(image)
    for cnt in contours:
        cv2.drawContours(mask, [cnt], 0, 255, -1)
    result = cv2.bitwise_and(image, image, mask=mask)
    cv2.imwrite("s2.png", result)
    outline = np.array([[-1, -1, -1],[-1,  8, -1],[-1, -1, -1]])
    outline *= 4
    blur = np.array([[0.0625, 0.125, 0.0625],[0.125,  0.25,  0.125],[0.0625, 0.125, 0.0625]])
    blur *= 4
    sharpen = np.array([[0, -1, 0],[-1, 8, -1],[0, -1, 0]])
    def convolve(img: np.array, kernel: np.array) -> np.array:
        size = (img.shape[0] - kernel.shape[0], img.shape[1] - kernel.shape[1])
        k = kernel.shape[0]
        convolved_img = np.zeros(shape=(size[0], size[1]))
        for i in range(size[0]):
            for j in range(size[1]):
                mat = img[i:i+k, j:j+k]
                convolved_img[i, j] = np.sum(np.multiply(mat, kernel)) 
        return convolved_img
    processed = image.astype(np.int64)
    s = (len(processed[0]), len(processed))
    im = Image.new(mode="RGB", size=s)
    pix = im.load()
    for i in range(s[0]):
        for j in range(s[1]):
            num = int(processed[j][i])
            pix[i,j] = (num, num, num)
    im.save('s3.png')

    # Size Changes
    image = cv2.imread('s3.png')
    image = cv2.GaussianBlur(image,(9,9),0)
    src = image
    scale_percent = 75
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)
    dsize = (width, height)
    output = cv2.resize(src, dsize)
    cv2.imwrite('s4.png',output)

    # Final Noise Reduction
    image = cv2.imread('s4.png')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 5500:
            cv2.drawContours(thresh, [c], -1, (0,0,0), -1)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    close = 255 - cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    cv2.imwrite('s5.png', thresh)
    cv2.imwrite('s6.png', close)

    # Smoothen
    image = Image.open('s6.png')
    image = image.filter(ImageFilter.ModeFilter(size=30))
    image.save('PROCESSED_FINAL.png')

    # Flood Fill Edges
    # image = cv2.floodFill(image, )




    # Source: (https://learnopencv.com/filling-holes-in-an-image-using-opencv-python-c/)

    # Floodfill Image on Edges
    im_in = cv2.imread("PROCESSED_FINAL.png", cv2.IMREAD_GRAYSCALE)
    th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
    im_floodfill = im_th.copy()

    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    
    for i in range(w):
        # Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (i,0), 255, 0, 2)

        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

    for i in range(w):
        # Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (i,h-1), 255, 0, 2)
        
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

    for i in range(h):
        # Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (0,i), 255, 0, 2)

        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

    for i in range(h):
        # Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (w-1,i), 255, 0, 2)
        
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

    cv2.imwrite('s7.png', im_floodfill_inv)





    import skimage
    import imageio.v3 as iio

    image = iio.imread('s7.png')[:,:]

    image = image.copy()

    out = skimage.morphology.area_opening(image, area_threshold=13500, connectivity=2)

    cv2.imwrite('s8.png', out)
    cv2.imwrite('PROCESSED_FINAL_2.png', out)



    import numpy as np
    import cv2
    import random

    for k in range(2):
        img = cv2.imread('PROCESSED_FINAL_2.png', 0)
        img = cv2.bitwise_not(img)
        original_img = cv2.imread('images/sample' + str(sample) + '.tif')  # For OPERATION
        # original_img = cv2.imread('output.png')  # For TESTING

        src = img
        scale_percent = 100 / 0.75
        width = int(src.shape[1] * scale_percent / 100)
        height = int(src.shape[0] * scale_percent / 100)
        dsize = (width, height)
        img = cv2.resize(src, dsize)

        _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        ################# Expand contours test
        scalefactor = 1.1
        for cnt in contours:
            M = cv2.moments(cnt)
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])

            for point in cnt:
                dist = np.sqrt((point[0][0] - cx)**2 + (point[0][1] - cy)**2)
                scaled_dist = dist * scalefactor
                angle = np.arctan2(point[0][1] - cy, point[0][0] - cx)
                newx = int(cx + scaled_dist * np.cos(angle))
                newy = int(cy + scaled_dist * np.sin(angle))
                point[0][0] = newx
                point[0][1] = newy
        #################

        output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        font = cv2.FONT_HERSHEY_COMPLEX

        for j, contour in enumerate(contours):
            color = tuple(random.randint(0, 255) for _ in range(3))

            if k == 1:
                contour = cv2.convexHull(contour)  # For smoothing test

            cv2.drawContours(output, [contour], 0, color, 2)
            cv2.drawContours(original_img, [contour], 0, color, 2)

            moments = cv2.moments(contour)
            centroid_x = int(moments["m10"] / moments["m00"])
            centroid_y = int(moments["m01"] / moments["m00"])

            area = cv2.contourArea(contour) / 4.105
            diameter = np.sqrt(4 * area / np.pi)

            # text = f"Size: {area:.2f} pixels\nDiameter: {diameter:.2f} pixels"  # Actual
            text = f"{area:.0f}um (Area)"  # Test, the .0f number of 0 means round to 0 decimals
            diameter_text = f"{diameter:.0f}um (Dia)"  # Test, the .0f number of 0 means round to 0 decimals
            text_size, _ = cv2.getTextSize(text, font, 0.6, 1)
            diameter_text_size, _ = cv2.getTextSize(diameter_text, font, 0.6, 1)

            x = centroid_x - text_size[0] // 2
            y = centroid_y - text_size[1] // 2
            xd = centroid_x - diameter_text_size[0] // 2
            yd = centroid_y - diameter_text_size[1] // 2

            cv2.putText(output, text, (x, y - 15), font, 0.7, (200, 200, 200), 2)
            cv2.putText(original_img, text, (x, y - 15), font, 0.7, (0, 0, 0), 2)
            cv2.putText(output, diameter_text, (xd, yd + 15), font, 0.7, (200, 200, 200), 2)
            cv2.putText(original_img, diameter_text, (xd, yd + 15), font, 0.7, (0, 0, 0), 2)

        cell_num = len(contours)
        text = str(cell_num) + " Adipocytes"
        text_size = cv2.getTextSize(text, font, 3, 2)[0]
        # font = cv2.FONT_HERSHEY_SIMPLEX
        font = cv2.FONT_HERSHEY_COMPLEX

        x = 50
        y = original_img.shape[0] - text_size[1] - 10
        cv2.putText(original_img, text, (x, y), font, 3, (0, 0, 0), 2)

        output = cv2.bitwise_not(output)
        cv2.imwrite('PROCESSED_FINAL_3.png', output)
        cv2.imwrite('PROCESSED_FINAL_3.1.png', original_img)
        if k == 0:
            cv2.imwrite("output/output" + str(image_test_saving) + ".png", original_img)
        else:
            cv2.imwrite("output/smooth_output" + str(image_test_saving) + ".png", original_img)

images/sample1.tif
